In [24]:
import pandas as pd
import sys
import math

In [25]:
# (price range, box size)
box_ranges = [(.25,.0625),
              (1,.125), 
              (5,.25),
              (20,.5),
              (100,1),
              (200,2),
              (500,4),
              (1000,5),
              (25000,50),
              (sys.maxsize,500)]


In [26]:
def updateBoxSize(price):
    
    for i in range (len(box_ranges)):
        if price < box_ranges[i][0]:
            return box_ranges[i][1]
    return None

#     if XO == 'X':
#         for i in range(len(box_ranges)):
#             if high < box_ranges[i][1]:
#                 return box_sizes[i]
#     else:
#         for i in range(len(box_ranges)):
#             if low < box_ranges[i][1]:
#                 return box_sizes[i]
            
#     # should never get to here but just to be safe
#     return None# returns 2

In [27]:
# going to assume the scale starts at 0
# this is going to return the value of the box that is rounded down to the nearest box_size
def updateBoxPrice(price, box_size, XO):
    if XO == 'X':
        if box_size >= 1:
            return price - (price % box_size)
        else:
            inter = math.ceil(price)
            while inter > price:
                inter = inter - box_size
            return inter
    else:
        if box_size >= 1:
            return price - (price % box_size) + box_size
        else:
            inter = int(price)
            while inter < price:
                inter = inter + box_size
            return inter
# def updateBoxPriceO(high, low, XO, box_size, adj_close):

#     return adj_close - (adj_close % box_size)+box_size
        
        
#     if XO == 'X':
#         mod_val = high % box_size
#         return high - mod_val
#     else:
#         mod_val = low % box_size
#         return low - mod_val

In [28]:
def add_boxes(old_price, new_price, box_size, XO):
    
    boxes_diff = updateBoxPrice(old_price, box_size, XO) - updateBoxPrice(new_price, box_size, XO)
    #print(str(inter/box_size))
    return abs(int(boxes_diff/box_size))
    

In [29]:
def column_X (reversal, df, row):
    # try adding an X with the high price.
    # if can't add an X:
    # try low and if it creates a 3 box reversal, then use that
    # otherwise do nothing
    box_size = updateBoxSize(row['high'])
    box_price = updateBoxPrice(row['high'], box_size, 'X')
    
  #  added_boxes = add_boxes(row['high'], df[-1]['high'], box_size, 'X')
  #  sub_boxes = add_boxes(df[-1]['low'], row['low'], box_size, 'X')          
    
    if row['high'] >= df[-1]['box_high']+box_size:
       # print(row['high'])
        df[-1]['box_high'] = updateBoxPrice(row['high'],box_size,'X')
        
        df[-1]['high'] = row['high']               
        
#         if row['low'] < df[-1]['low']: 
#             df[-1]['low'] = row['low']       
        
        df[-1]['date_end'] = row['date']
        df[-1]['day'][row['date']] = {'box_high': df[-1]['box_high'], 
                                    'box_low' : df[-1]['box_low'],
                                      'high':row['high'],
                                      'low':row['low'],
                                      'close':row['close'],
                                         'box_size':box_size}
        
    elif row['low'] <= df[-1]['box_high']-((reversal)*box_size):
         
        # creating a new column of Os        
        df_temp = {'date_start':row['date'],
                    'date_end':row['date'],
                    'XO':'O', # the high and low shouldn't matter because we are comparing box prices
                    'high': row['high'],
                    'low': row['low'],
                    'box_high': (df[-1]['box_high']-box_size),# here we need to append the boxes to the O column to start at the box right below the highest box in the X column
                    'box_low': updateBoxPrice(row['low'],updateBoxSize(row['low']),'O'),
                    'day':{row['date']:
                        {'box_high':(df[-1]['box_high']-box_size), 
                        ""'box_low' : updateBoxPrice(row['low'],updateBoxSize(row['low']),'O'),
                        'high':row['high'],
                         'low':row['low'],
                         'close':row['close'],
                        'box_size':box_size}}}
        df.append(df_temp)
    else:
        # if no movement in column
        df[-1]['date_end'] = row['date']
        df[-1]['day'][row['date']] = {'box_high': df[-1]['box_high'],
                                      'box_low' : df[-1]['box_low'],
                                      'high':row['high'],
                                      'low':row['low'],
                                      'close':row['close'],
                                     'box_size' : box_size}
        
    return df

In [30]:
def column_O (reversal, df, row):
    # use the low if you can create a new O box
    # use the high when another O can't be drawn and we creates a 3 box reversal
    # ignore else
    
    box_size = updateBoxSize(row['close'])
    box_price = updateBoxPrice(row['close'], box_size, 'O')
 
    sub_boxes = add_boxes(row['low'],df[-1]['low'], box_size, 'O')
    added_boxes = add_boxes(row['high'],df[-1]['high'], box_size, 'O')    
#    print(added_boxes)
#    print(sub_boxes)  
#     print("O added_boxes: " + str(added_boxes))
#     print("O sub_boxes: " + str(sub_boxes))    
#     print('O box_high: ' + (str(df[-1]['box_high'])))
#     print('O box_low: ' + (str(df[-1]['box_low'])))
#     print('O high: ' + str(df[-1]['high']))    
#     print('O low: ' + str(df[-1]['low']))    
#     print('O date: ' +row['date']) 
    
    if row['low'] <= df[-1]['box_low']-box_size:


        df[-1]['box_low'] = updateBoxPrice(row['low'],box_size,'O')

        df[-1]['low'] = row['low']
        
#         if row['high'] > df[-1]['high']: 
#             df[-1]['high'] = row['high']
        
        df[-1]['date_end'] = row['date']
        
        df[-1]['day'][row['date']] = {'box_high': df[-1]['box_high'],
                                       'box_low' : df[-1]['box_low'],
                                      'high':row['high'],
                                      'low':row['low'],
                                      'close':row['close'],
                                         'box_size':box_size}
        
    elif row['high'] >= df[-1]['box_low']+((reversal)*box_size):


        df_temp = {'date_start':row['date'],
                    'date_end':row['date'],
                    'XO':'X', # the high and low shouldn't matter because we are comparing box prices
                    'high': row['high'],
                    'low': row['low'],
                    'box_high': updateBoxPrice(row['high'],updateBoxSize(row['high']), 'X'), # here we need to append the boxes to the O column to start at the box right below the highest box in the X column
                    'box_low': df[-1]['box_low'] + box_size,
                    'day':{row['date']:
                        {'box_high':updateBoxPrice(row['high'],updateBoxSize(row['high']), 'X'),
                         'box_low' : df[-1]['box_low'] + box_size,
                        'high':row['high'],
                         'low':row['low'],
                         'close':row['close'],
                            'box_size':box_size}}}
        df.append(df_temp)
    else:
        # if no movement in column
        df[-1]['date_end'] = row['date']
        df[-1]['day'][row['date']] = {'box_high': df[-1]['box_high'],
                                      'box_low' : df[-1]['box_low'],
                                      'high':row['high'],
                                      'low':row['low'],
                                      'close':row['close'],
                                         'box_size':box_size}
    return df
        

In [31]:

def constructPFChart(data):
    # starting out, we assume that the first box is an X

    box_size = updateBoxSize(data['high'].iloc[0])
    box_price = updateBoxPrice(data['high'].iloc[0], box_size, 'X')
    
    reversal = 3
    df = []
    df_temp = {'date_start':data['date'].iloc[0], 
           'date_end':data['date'].iloc[0],
           'XO':'X',
           'high':data['high'].iloc[0],
           'low':data['low'].iloc[0],
            'box_high':box_price,
            'box_low':box_price-((reversal-1)*box_size),
            'day': {data['date'].iloc[0]:{'box_high': box_price, 
                                     'box_low' : box_price-((reversal-1)*box_size),
                                     'high':data['high'].iloc[0], 
                                     'low':data['low'].iloc[0], 
                                     'close':data['close'].iloc[0],
                                         'box_size':box_size}}}
    df.append(df_temp)

    for index, row in data.iloc[1:].iterrows():
        #print(row)
        #print()
        #print(df)
        #print()        
        # check if the previous day's column is an X or an O
        if df[-1]['XO'] == 'X':
           # print('X')
           # print(row)
            df = column_X(reversal, df, row)
        else:
           # print('O')
           # print(row)            
            df = column_O(reversal, df, row)

    return df

In [32]:
# need to replace last box price with highest and lowest box price
def get_ratings(pf_data, query_data):
    


    # need to first get start and end index
    start_column = 0
    end_column = len(pf_data)
   # print(pf_data[0])
    for i in range(len(pf_data)):
        if query_data.iloc[0]['date'] in pf_data[i]['day']:
            start_column = i
            break
            
    for i in range(start_column, len(pf_data)):
        if query_data.iloc[-1]['date'] in pf_data[i]['day']:
            end_column = i
            break
              
    # now we get the ratings
    # if X, then look at previous X's column
    # if current price is higher than previous X's column's high
    # that is a 1
    # if in a O column, and you go below the previous O column's low,
    # that is a sell mode with rating 4
    # if you're in a sell mode and you move to a column of X's, that is a rating of 3
    # if you're in a buy mode and you move into a column of O's that is a rating of 2
    # going through each date of the query data
    ratings = []
    boxes = []
    XOs = []
    for index, row in query_data.iterrows():
#        print(row['date'])
        # finding where it is in the pf chart

        while row['date'] not in pf_data[start_column]['day']:

            start_column += 1
            
#        print(pf_data[start_column]['day'][row['date']]['box_price'])
#        print(row)
        if pf_data[start_column]['XO'] == 'X':
            boxes.append(pf_data[start_column]['day'][row['date']]['box_high'])
        else:
            boxes.append(pf_data[start_column]['day'][row['date']]['box_low'])
        XOs.append(pf_data[start_column]['XO'])
        
        if pf_data[start_column]['XO'] == 'X':
            # Note: clearly 2 columns back is also an X  (gotta be careful abt -2 if we started at 0 or 1)              
            
            if boxes[-1] > pf_data[start_column - 2]['box_high']:
                ratings.append(1)
            else:
                if len(ratings) == 0:
                    ratings.append(3)
                elif ratings[-1] == 1:
                    ratings.append(1)
                else:
                    ratings.append(3)
        else:

            if boxes[-1] < pf_data[start_column - 2]['box_low']:
                ratings.append(4)
            else:
                if len(ratings) == 0:
                    ratings.append(2)
                elif ratings[-1] == 4:
                    ratings.append(4)
                else:
                    ratings.append(2)
  #  print(len(ratings))
  #  print(len(boxes))
    query_data['PF_rating'] = ratings
    query_data['box_price'] = boxes
    query_data['XO'] = XOs

    return query_data

In [33]:
def get_PF_Ratings(full_data, asset):

    pf_data = constructPFChart(full_data)
    

    full_data['asset'] = asset 
    return get_ratings(pf_data, full_data)


In [38]:
df = pd.read_csv('data/AAPL.csv')


In [39]:
df[4900:5000]

,Unnamed: 0,date,open,high,low,close
4900,4900,06-10-2009,17.87,17.88,17.37,17.62
4901,4901,06-11-2009,17.53,17.78,17.41,17.58
4902,4902,06-12-2009,17.44,17.48,17.09,17.21
4903,4903,06-15-2009,17.09,17.20,16.95,17.10
4904,4904,06-16-2009,17.17,17.40,17.10,17.13
4905,4905,06-17-2009,17.17,17.27,16.90,17.03
4906,4906,06-18-2009,17.10,17.34,17.03,17.07
4907,4907,06-19-2009,17.35,17.53,17.20,17.52
4908,4908,06-22-2009,17.67,17.78,17.13,17.26
4909,4909,06-23-2009,17.14,17.21,16.69,16.84


In [40]:
d = get_PF_Ratings(df,"AAPL")

In [41]:
#pf_chart = constructPFChart(df)
#d[200:]
d[4900:7500]

,Unnamed: 0,date,open,high,low,close,asset,PF_rating,box_price,XO
4900,4900,06-10-2009,17.87,17.88,17.37,17.62,AAPL,1,18.0,X
4901,4901,06-11-2009,17.53,17.78,17.41,17.58,AAPL,1,18.0,X
4902,4902,06-12-2009,17.44,17.48,17.09,17.21,AAPL,1,18.0,X
4903,4903,06-15-2009,17.09,17.20,16.95,17.10,AAPL,1,18.0,X
4904,4904,06-16-2009,17.17,17.40,17.10,17.13,AAPL,1,18.0,X
4905,4905,06-17-2009,17.17,17.27,16.90,17.03,AAPL,1,18.0,X
4906,4906,06-18-2009,17.10,17.34,17.03,17.07,AAPL,1,18.0,X
4907,4907,06-19-2009,17.35,17.53,17.20,17.52,AAPL,1,18.0,X
4908,4908,06-22-2009,17.67,17.78,17.13,17.26,AAPL,1,18.0,X
4909,4909,06-23-2009,17.14,17.21,16.69,16.84,AAPL,1,18.0,X


In [70]:
#len(pf_chart) #150
#pf_chart[211]

In [71]:
#pf_chart[300]

In [77]:
#df = pd.read_csv('data/V.csv')

In [84]:
#d = get_PF_Ratings('05-02-2008', '01-01-2012', 'd', "V",df)

           date   high    low  close
32   05-02-2008  20.24  18.73  19.14
33   05-05-2008  19.78  18.69  19.67
34   05-06-2008  20.39  19.60  20.25
35   05-07-2008  20.78  20.28  20.47
36   05-08-2008  20.76  19.88  20.19
37   05-09-2008  20.05  19.09  19.17
38   05-12-2008  19.54  18.06  18.94
39   05-13-2008  19.31  18.69  19.05
40   05-14-2008  19.51  18.84  19.02
41   05-15-2008  19.18  18.76  19.02
42   05-16-2008  19.16  18.79  19.05
43   05-19-2008  19.60  19.00  19.27
44   05-20-2008  19.39  18.95  19.14
45   05-21-2008  19.30  18.62  18.70
46   05-22-2008  18.64  17.62  17.88
47   05-23-2008  17.88  17.37  17.66
48   05-27-2008  18.26  17.20  18.21
49   05-28-2008  18.95  18.40  18.95
50   05-29-2008  20.11  19.20  19.72
51   05-30-2008  20.25  19.85  19.97
52   06-02-2008  20.11  19.57  19.91
53   06-03-2008  20.34  19.90  20.25
54   06-04-2008  20.38  19.72  19.77
55   06-05-2008  20.18  19.80  20.18
56   06-06-2008  19.89  19.32  19.35
57   06-09-2008  19.65  18.63  19.00
5

In [86]:
#d[0:50]

,date,high,low,close,asset,PF_rating,box_price,XO
32,05-02-2008,20.24,18.73,19.14,V,3,20.0,X
33,05-05-2008,19.78,18.69,19.67,V,3,20.0,X
34,05-06-2008,20.39,19.60,20.25,V,3,20.0,X
35,05-07-2008,20.78,20.28,20.47,V,3,20.0,X
36,05-08-2008,20.76,19.88,20.19,V,3,20.0,X
37,05-09-2008,20.05,19.09,19.17,V,3,20.0,X
38,05-12-2008,19.54,18.06,18.94,V,2,18.5,O
39,05-13-2008,19.31,18.69,19.05,V,2,18.5,O
40,05-14-2008,19.51,18.84,19.02,V,2,18.5,O
41,05-15-2008,19.18,18.76,19.02,V,2,18.5,O
